In [1]:
import os

import tensorflow as tf

from lib.dataset.parse_heightmap import parse_heightmap
from lib.ai.model_rainfallwater_mono import model_rainfallwater_mono
from lib.ai.helpers.make_callbacks import make_callbacks
from lib.ai.helpers.summarywriter import summarywriter

2022-12-09 15:49:31.909879: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 15:49:31.909893: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
filepath_heightmap="/mnt/research-data/main/terrain50-nimrodsized.json.gz"

dir_output = "/tmp/x/mono_segment_TEST"
if not os.path.exists(os.path.join(dir_output, "checkpoints")):
	os.makedirs(os.path.join(dir_output, "checkpoints"))

In [3]:
model = model_rainfallwater_mono(
	metadata={ "rainfallradar": [ 1, 64, 64 ] },
	model_arch_dec="convnext_i_xxtiny"
)

summarywriter(model, filepath_output=os.path.join(dir_output, "summary.txt"))

RAINFALL channels 1 width 64 height 64 HEIGHTMAP_INPUT False
convnext:shape IN x (None, 64, 64, 1)
DEBUG:convnext shape x (None, 64, 64, 1)


2022-12-09 15:49:34.026632: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-12-09 15:49:34.026670: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: SIMVIS-CO45428A
2022-12-09 15:49:34.026678: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: SIMVIS-CO45428A
2022-12-09 15:49:34.026772: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2022-12-09 15:49:34.026795: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.85.2
2022-12-09 15:49:34.026801: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 510.85.2 does not match DSO version 510.108.3 -- cannot find working devices in this configuration
2022-12-09 15:49:34.027072: I tensorflow/core/platform/cpu_feature_guard.cc:193]

DEBUG:model ENCODER output_shape (None, 512)
DEBUG:model BOTTLENECK:stack2image output_shape (None, 4, 4, 512)


2022-12-09 15:49:37.559 | WARNING  | lib.ai.model_rainfallwater_mono:model_rainfallwater_mono:70 - Warning: TODO implement attention from https://ieeexplore.ieee.org/document/9076883
2022-12-09 15:49:37.613 | INFO     | lib.ai.model_rainfallwater_mono:model_rainfallwater_mono:80 - learning_rate: None


In [4]:
heightmap = parse_heightmap(filepath_heightmap) / 100
heightmap = tf.image.crop_to_bounding_box(tf.expand_dims(heightmap, axis=-1), 0, 0, 64, 64)
heightmap_labels = tf.one_hot(tf.cast(tf.math.greater(tf.squeeze(heightmap)/10, 0.05), dtype=tf.int32), 2)

dataset = tf.data.Dataset.from_tensor_slices([heightmap])
dataset_labels = tf.data.Dataset.from_tensor_slices([heightmap_labels])

for item in dataset_labels:
	print("cells", 64*64, "cells/2", (64*64)/2, "shape+", item.shape, tf.math.reduce_sum(tf.math.argmax(item, axis=-1)))
	break
dataset = tf.data.Dataset.zip((dataset, dataset_labels)) \
	.repeat(64 * 64) \
	.batch(64) \
	.prefetch(tf.data.AUTOTUNE)


cells 4096 cells/2 2048.0 shape+ (64, 64, 2) tf.Tensor(2401, shape=(), dtype=int64)


In [5]:
model.fit(
	dataset,
	epochs=50,
	callbacks=make_callbacks("/tmp/x/mono_segment_TEST", model)
)

Epoch 1/50


In [ ]:
prediction = model.predict(tf.expand_dims(heightmap, axis=0))
print(tf.math.argmax(prediction, axis=-1))
print(tf.math.reduce_sum(tf.math.argmax(prediction, axis=-1)))
print(prediction)
